# Neural Machine Translation by Jointly Learning to Align and Translate

In this third notebook on sequence-to-sequence models using PyTorch and TorchText, we'll be implementing the model from [Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473) to building an language to language transformation model using the collection of copyright free books aligned by Andras Farkas, which are available from http://www.farkastranslations.com/bilingual_books.php

## Introduction

Here is the general encoder-decoder model:

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq1.png?raw=1)

In the previous model, our architecture was set-up in a way to reduce "information compression" by explicitly passing the context vector, $z$, to the decoder at every time-step and by passing both the context vector and embedded input word, $d(y_t)$, along with the hidden state, $s_t$, to the linear layer, $f$, to make a prediction.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq7.png?raw=1)

Even though we have reduced some of this compression, our context vector still needs to contain all of the information about the source sentence. The model implemented in this notebook avoids this compression by allowing the decoder to look at the entire source sentence (via its hidden states) at each decoding step! How does it do this? It uses *attention*. 

Attention works by first, calculating an attention vector, $a$, that is the length of the source sentence. The attention vector has the property that each element is between 0 and 1, and the entire vector sums to 1. We then calculate a weighted sum of our source sentence hidden states, $H$, to get a weighted source vector, $w$. 

$$w = \sum_{i}a_ih_i$$

We calculate a new weighted source vector every time-step when decoding, using it as input to our decoder RNN as well as the linear layer to make a prediction. We'll explain how to do all of this during the session.


## Import Statements

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.datasets import Multi30k
from torchtext import data
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time
import json

Set the random seeds for reproducability.

In [3]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [4]:
# downloading spacy english and italian
# !python3 -m spacy download it_core_news_sm
# !python -m spacy download en_core_web_sm

loading english and italian spacy models

In [5]:
spacy_it = spacy.load("it_core_news_sm")
spacy_en = spacy.load("en_core_web_sm")

## Previewing the Dataset

Link to the dataset

http://opus.nlpl.eu/Books.php

http://opus.nlpl.eu/

In [6]:
# using translate-toolkit to use tmx file
# !pip3 install translate-toolkit

In [7]:
from translate.storage.tmx import tmxfile
with open("./../datasets/Multilingual Datasets for Chatbot Training/en-it.tmx", 'rb') as fin:
    tmx_file = tmxfile(fin, 'en', 'it')

In [8]:
language_pair_list = []
for node in tmx_file.unit_iter():
    language_pair_list.append([node.source, node.target])
len(language_pair_list)

32035

In [9]:
language_pair_list[5]

['We had been wandering, indeed, in the leafless shrubbery an hour in the morning; but since dinner (Mrs. Reed, when there was no company, dined early) the cold winter wind had brought with it clouds so sombre, and a rain so penetrating, that further out-door exercise was now out of the question.',
 "La mattina avevamo errato per un'ora nel boschetto spogliato di foglie, ma dopo pranzo (quando non vi erano invitati, la signora Reed desinava presto), il vento gelato d'inverno aveva portato seco nubi così scure e una pioggia così penetrante, che non si poteva pensare a nessuna escursione."]

## Defining the fields

Question and Answer are standard Field object, where we have decided to use the spaCy tokenizer and convert all the text to lower‐ case.

In [10]:
English = Field(sequential = True,
                      tokenize = spacy_en,
                      include_lengths=True,
                      init_token = '<sos>',
                      eos_token = '<eos>')
#                       lower = True)

Italian = Field(sequential = True,
                    tokenize =spacy_it,
                    include_lengths=True,
                    init_token = '<sos>',
                    eos_token = '<eos>')
#                     lower = True)

/home/tensor/.local/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Having defined those fields, we now need to produce a list that maps them onto the list of rows that are in the list quescontext_ans_pairs

In [11]:
fields = [('english', English),('italian',Italian)]


In [12]:
example = [data.Example.fromlist([language_pair_list[i][0],language_pair_list[i][1]], fields) for i in range(len(language_pair_list))] 

/home/tensor/.local/lib/python3.8/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


## Creating dataset

In [13]:
# creating dataset
LangDataset = data.Dataset(example, fields)

Finally, we can split into training, testing, and validation sets by using the split() method:

In [14]:
(train, valid, test) = LangDataset.split(split_ratio=[0.70, 0.15, 0.15], random_state=random.seed(SEED))

In [15]:
(len(train), len(valid), len(test))

(22424, 4806, 4805)

An example from the dataset:

In [16]:
vars(train.examples[10])

{'english': When can we meet?', 'italian': Quando ci vediamo?}

## Data Augmentation
copied from the repository of Woncoh1 https://github.com/woncoh1/END/blob/main/Assignment_Session_7_Sentiment_Analysis_using_LSTM_RNN.ipynb

In [17]:
# import libraries and prepare sentence text

import random
# for back translation
import google_trans_new
from google_trans_new import google_translator
# for word tokenization
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /home/tensor/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### Back Translation

In [18]:
# translate a sentence to a random language,
# and translate back to original language

def back_translate(sentence, p=0.1, lang="en"):
    sentence = [str(i) for i in sentence]
    # do nothing with probability of 1-p
    if random.uniform(0,1) > p:
        return sentence
    
    # combine tokenized sentence into one string
    sentence = " ".join(sentence)
    
    # instantiate translator
    translator = google_translator()
    
    # choose a target language
    available_langs = list(google_trans_new.LANGUAGES.keys())
    trans_lang = random.choice(available_langs)
    
    # translate to the target language
    translations = translator.translate(sentence, lang_tgt=trans_lang)
    
    # translate back to original language
    translations_en_random = translator.translate(translations, lang_src=trans_lang, lang_tgt=lang)
    
    # select only one translation
    if len(translations_en_random) > 1:
        translations_en_random = translations_en_random[0]
        
    return word_tokenize(translations_en_random)

#### Random Deletion

In [19]:
# randomly delete words from a sentence with a given probability

def random_deletion(sentence, p=0.5): 
    # return if single word
    if len(sentence) == 1: 
        return sentence
    # delete words
    remaining = list(filter(lambda x: random.uniform(0,1) > p, sentence)) 
    # if nothing left, sample a random word
    if len(remaining) == 0: 
        return [random.choice(sentence)] 
    else:
        return remaining

#### Random Swap

In [20]:
# randomly swap a pair of words in a sentence for a given # of times

def random_swap(sentence, n=5): 
    if len(sentence) < 2:
        return sentence
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

#### Carry Out Data Augmentation

In [22]:
for idx, example in enumerate(train.examples):
    example.english = back_translate(example.english, p=0.01, lang="en")
    example.english = random_deletion(example.english, p=0.1)
    example.english = random_swap(example.english, n=1)
    example.italian = back_translate(example.italian, p=0.01, lang="it")
    example.italian = random_deletion(example.italian, p=0.1)
    example.italian = random_swap(example.italian, n=1)

## Building Vocabulary

In [23]:
import torchtext.vocab as vocab

In [24]:
english_embeddings = vocab.Vectors(name = './wiki.simple.vec')
italian_embeddings = vocab.Vectors(name = './cc.it.300.vec')

In [25]:
MAX_VOCAB_SIZE = 20000
''' Use max of 25000 frequently occuring words to create vocabulary. Use pretrained fasttext embedding. Embedding of words that exist in the data but not in the
dictionary are set using normal distribution.
'''
English.build_vocab(train,
                        max_size = MAX_VOCAB_SIZE,
                        vectors = english_embeddings,
                        unk_init = torch.Tensor.normal_,
                        min_freq = 2)
Italian.build_vocab(train,
                        max_size = MAX_VOCAB_SIZE,
                        vectors = italian_embeddings,
                        unk_init = torch.Tensor.normal_,
                        min_freq = 2)

In [26]:
# get the vocab instances
english_vocab = English.vocab
italian_vocab = Italian.vocab

english_embedding_vector = english_vocab.vectors
italian_embedding_vector = italian_vocab.vectors
print(english_embedding_vector.shape)
print(italian_embedding_vector.shape)

torch.Size([12116, 300])
torch.Size([16584, 300])


By default, torchtext will add two more special tokens, <unk> for unknown words and <pad>, a padding token that will be used to pad all our text to roughly the same size to help with efficient batching on the GPU.

In [27]:
print('Size of english vocab : ', len(english_vocab))
print('Size of italian vocab : ', len(italian_vocab))
print('Top 10 words appeared in english:', list(english_vocab.freqs.most_common(10)))
print("Top 10 words appeared in italian:", list(italian_vocab.freqs.most_common(10)))

Size of english vocab :  12116
Size of italian vocab :  16584
Top 10 words appeared in english: [(',', 36744), ('the', 20803), ('.', 17881), ('and', 16945), ('to', 13794), ('I', 10967), ('of', 10671), ("'", 9838), ('a', 9095), ('in', 6609)]
Top 10 words appeared in italian: [(',', 34937), ('.', 19067), ('e', 13942), ('di', 12820), ('che', 10912), ('—', 10114), ('la', 7980), ('non', 6880), ('a', 6741), ('il', 6494)]


## Create the iterators

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [29]:
BATCH_SIZE = 8
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits((train, valid, test),
                                                            batch_size = BATCH_SIZE, 
                                                            sort_key = lambda x: len(x.english),
                                                            sort_within_batch=True,
                                                            device = device)

/home/tensor/.local/lib/python3.8/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [30]:
vars(English.vocab)

{'freqs': Counter({'Left': 6,
          'themselves': 111,
          'mouths': 7,
          'the': 20803,
          'children': 184,
          'started': 64,
          'milk': 27,
          'over': 592,
          'a': 9095,
          'candle': 42,
          ',': 36744,
          'and': 16945,
          'jets': 1,
          'cooking': 6,
          'into': 831,
          '.': 17881,
          "'": 9838,
          'He': 1290,
          'that': 6235,
          'married': 90,
          'careth': 2,
          'for': 3450,
          'things': 314,
          'are': 1111,
          'world': 181,
          'how': 615,
          'he': 5821,
          'may': 293,
          'please': 88,
          'his': 4386,
          'wife': 308,
          'but': 2969,
          'was': 6280,
          'is': 2770,
          'unmarried': 8,
          'belong': 9,
          'to': 13794,
          'Lord': 40,
          'Paul': 1,
          ';': 4815,
          'Karenin': 311,
          'who': 1079,
          'now': 

Save the vocabulary for later use

In [31]:
import os, pickle
with open('english_tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(english_vocab.stoi, tokens)
with open("italian_tokenizer.pkl", "wb") as tokens:
    pickle.dump(italian_vocab.stoi, tokens)

## Building the model

### Encoder

First, we'll build the encoder using a single layer GRU, however we now use a *bidirectional RNN*. With a bidirectional RNN, we have two RNNs in each layer. A *forward RNN* going over the embedded sentence from left to right (shown below in green), and a *backward RNN* going over the embedded sentence from right to left (teal). All we need to do in code is set `bidirectional = True` and then pass the embedded sentence to the RNN as before. 

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq8.png?raw=1)

We now have:

$$\begin{align*}
h_t^\rightarrow &= \text{EncoderGRU}^\rightarrow(e(x_t^\rightarrow),h_{t-1}^\rightarrow)\\
h_t^\leftarrow &= \text{EncoderGRU}^\leftarrow(e(x_t^\leftarrow),h_{t-1}^\leftarrow)
\end{align*}$$

Where $x_0^\rightarrow = \text{<sos>}, x_1^\rightarrow = \text{guten}$ and $x_0^\leftarrow = \text{<eos>}, x_1^\leftarrow = \text{morgen}$.

As before, we only pass an input (`embedded`) to the RNN, which tells PyTorch to initialize both the forward and backward initial hidden states ($h_0^\rightarrow$ and $h_0^\leftarrow$, respectively) to a tensor of all zeros. We'll also get two context vectors, one from the forward RNN after it has seen the final word in the sentence, $z^\rightarrow=h_T^\rightarrow$, and one from the backward RNN after it has seen the first word in the sentence, $z^\leftarrow=h_T^\leftarrow$.

The RNN returns `outputs` and `hidden`. 

`outputs` is of size **[src len, batch size, hid dim * num directions]** where the first `hid_dim` elements in the third axis are the hidden states from the top layer forward RNN, and the last `hid_dim` elements are hidden states from the top layer backward RNN. We can think of the third axis as being the forward and backward hidden states concatenated together other, i.e. $h_1 = [h_1^\rightarrow; h_{T}^\leftarrow]$, $h_2 = [h_2^\rightarrow; h_{T-1}^\leftarrow]$ and we can denote all encoder hidden states (forward and backwards concatenated together) as $H=\{ h_1, h_2, ..., h_T\}$.

`hidden` is of size **[n layers * num directions, batch size, hid dim]**, where **[-2, :, :]** gives the top layer forward RNN hidden state after the final time-step (i.e. after it has seen the last word in the sentence) and **[-1, :, :]** gives the top layer backward RNN hidden state after the final time-step (i.e. after it has seen the first word in the sentence).

As the decoder is not bidirectional, it only needs a single context vector, $z$, to use as its initial hidden state, $s_0$, and we currently have two, a forward and a backward one ($z^\rightarrow=h_T^\rightarrow$ and $z^\leftarrow=h_T^\leftarrow$, respectively). We solve this by concatenating the two context vectors together, passing them through a linear layer, $g$, and applying the $\tanh$ activation function. 

$$z=\tanh(g(h_T^\rightarrow, h_T^\leftarrow)) = \tanh(g(z^\rightarrow, z^\leftarrow)) = s_0$$

**Note**: this is actually a deviation from the paper. Instead, they feed only the first backward RNN hidden state through a linear layer to get the context vector/decoder initial hidden state. This doesn't seem to make sense to me, so we have changed it.

As we want our model to look back over the whole of the source sentence we return `outputs`, the stacked forward and backward hidden states for every token in the source sentence. We also return `hidden`, which acts as our initial hidden state in the decoder.

In [32]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, emb_vector, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
#         self.embedding = nn.Embedding(input_dim, emb_dim)
        self.emb_vector = emb_vector
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
#         embedded = self.dropout(self.embedding(src))
        embedded = self.emb_vector[src]
#         embedded = embedded.to(device)
        #embedded = [src len, batch size, emb dim]
    
        embedded = embedded.to(device)
        
        outputs, hidden = self.rnn(embedded)
                
        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden

### Attention

Next up is the attention layer. This will take in the previous hidden state of the decoder, $s_{t-1}$, and all of the stacked forward and backward hidden states from the encoder, $H$. The layer will output an attention vector, $a_t$, that is the length of the source sentence, each element is between 0 and 1 and the entire vector sums to 1.

Intuitively, this layer takes what we have decoded so far, $s_{t-1}$, and all of what we have encoded, $H$, to produce a vector, $a_t$, that represents which words in the source sentence we should pay the most attention to in order to correctly predict the next word to decode, $\hat{y}_{t+1}$. 

First, we calculate the *energy* between the previous decoder hidden state and the encoder hidden states. As our encoder hidden states are a sequence of $T$ tensors, and our previous decoder hidden state is a single tensor, the first thing we do is `repeat` the previous decoder hidden state $T$ times. We then calculate the energy, $E_t$, between them by concatenating them together and passing them through a linear layer (`attn`) and a $\tanh$ activation function. 

$$E_t = \tanh(\text{attn}(s_{t-1}, H))$$ 

This can be thought of as calculating how well each encoder hidden state "matches" the previous decoder hidden state.

We currently have a **[dec hid dim, src len]** tensor for each example in the batch. We want this to be **[src len]** for each example in the batch as the attention should be over the length of the source sentence. This is achieved by multiplying the `energy` by a **[1, dec hid dim]** tensor, $v$.

$$\hat{a}_t = v E_t$$

We can think of $v$ as the weights for a weighted sum of the energy across all encoder hidden states. These weights tell us how much we should attend to each token in the source sequence. The parameters of $v$ are initialized randomly, but learned with the rest of the model via backpropagation. Note how $v$ is not dependent on time, and the same $v$ is used for each time-step of the decoding. We implement $v$ as a linear layer without a bias.

Finally, we ensure the attention vector fits the constraints of having all elements between 0 and 1 and the vector summing to 1 by passing it through a $\text{softmax}$ layer.

$$a_t = \text{softmax}(\hat{a_t})$$

This gives us the attention over the source sentence!

Graphically, this looks something like below. This is for calculating the very first attention vector, where $s_{t-1} = s_0 = z$. The green/teal blocks represent the hidden states from both the forward and backward RNNs, and the attention computation is all done within the pink block.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq9.png?raw=1)

In [33]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention= [batch size, src len]
        
        return F.softmax(attention, dim=1)

### Decoder

Next up is the decoder. 

The decoder contains the attention layer, `attention`, which takes the previous hidden state, $s_{t-1}$, all of the encoder hidden states, $H$, and returns the attention vector, $a_t$.

We then use this attention vector to create a weighted source vector, $w_t$, denoted by `weighted`, which is a weighted sum of the encoder hidden states, $H$, using $a_t$ as the weights.

$$w_t = a_t H$$

The embedded input word, $d(y_t)$, the weighted source vector, $w_t$, and the previous decoder hidden state, $s_{t-1}$, are then all passed into the decoder RNN, with $d(y_t)$ and $w_t$ being concatenated together.

$$s_t = \text{DecoderGRU}(d(y_t), w_t, s_{t-1})$$

We then pass $d(y_t)$, $w_t$ and $s_t$ through the linear layer, $f$, to make a prediction of the next word in the target sentence, $\hat{y}_{t+1}$. This is done by concatenating them all together.

$$\hat{y}_{t+1} = f(d(y_t), w_t, s_t)$$

The image below shows decoding the first word in an example translation.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq10.png?raw=1)

The green/teal blocks show the forward/backward encoder RNNs which output $H$, the red block shows the context vector, $z = h_T = \tanh(g(h^\rightarrow_T,h^\leftarrow_T)) = \tanh(g(z^\rightarrow, z^\leftarrow)) = s_0$, the blue block shows the decoder RNN which outputs $s_t$, the purple block shows the linear layer, $f$, which outputs $\hat{y}_{t+1}$ and the orange block shows the calculation of the weighted sum over $H$ by $a_t$ and outputs $w_t$. Not shown is the calculation of $a_t$.

In [34]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, emb_vector, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
#         self.embedding = nn.Embedding(output_dim, emb_dim)
        self.emb_vector = emb_vector
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
#         embedded = self.dropout(self.embedding(input))
        embedded = self.emb_vector[input]
        embedded = embedded.to(device)
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0)

### Seq2Seq

In this model we don't have to have the encoder RNN and decoder RNN have the same hidden dimensions, however the encoder has to be bidirectional. This requirement can be removed by changing all occurences of `enc_dim * 2` to `enc_dim * 2 if encoder_is_bidirectional else enc_dim`. 

This seq2seq encapsulator is that the `encoder` returns both the final hidden state (which is the final hidden state from both the forward and backward encoder RNNs passed through a linear layer) to be used as the initial hidden state for the decoder, as well as every hidden state (which are the forward and backward hidden states stacked on top of each other). We also need to ensure that `hidden` and `encoder_outputs` are passed to the decoder. 

Briefly going over all of the steps:
- the `outputs` tensor is created to hold all predictions, $\hat{Y}$
- the source sequence, $X$, is fed into the encoder to receive $z$ and $H$
- the initial decoder hidden state is set to be the `context` vector, $s_0 = z = h_T$
- we use a batch of `<sos>` tokens as the first `input`, $y_1$
- we then decode within a loop:
  - inserting the input token $y_t$, previous hidden state, $s_{t-1}$, and all encoder outputs, $H$, into the decoder
  - receiving a prediction, $\hat{y}_{t+1}$, and a new hidden state, $s_t$
  - we then decide if we are going to teacher force or not, setting the next input as appropriate

In [35]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

## Training the Seq2Seq Model

We initialise our parameters, encoder, decoder and seq2seq model (placing it on the GPU if we have one). 

In [36]:
INPUT_DIM = len(english_vocab)
OUTPUT_DIM = len(italian_vocab)
ENC_EMB_DIM = 300
DEC_EMB_DIM = 300
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, english_embedding_vector, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, italian_embedding_vector, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

We use a simplified version of the weight initialization scheme used in the paper. Here, we will initialize all biases to zero and all weights from $\mathcal{N}(0, 0.01)$.

In [37]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (rnn): GRU(300, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (rnn): GRU(1324, 512)
    (fc_out): Linear(in_features=1836, out_features=16584, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

Calculate the number of parameters

In [38]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 37,100,840 trainable parameters


We create an optimizer.

In [39]:
optimizer = optim.Adam(model.parameters())

We initialize the loss function.

In [40]:
it_PAD_IDX = italian_vocab.stoi[Italian.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = it_PAD_IDX)

We then create the training loop...

In [41]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src, src_length = batch.english
        trg, trg_length = batch.italian
        

        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
        if(i % 1000 == 0):
            print(f"{i} steps are done")
        
    return epoch_loss / len(iterator)

...and the evaluation loop, remembering to set the model to `eval` mode and turn off teaching forcing.

In [45]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src, src_length = batch.english
            trg, trg_length = batch.italian

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

Finally, define a timing function.

In [46]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Then, we train our model, saving the parameters that give us the best 


In [47]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'squad-attention.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

0 steps are done
1000 steps are done
2000 steps are done
Epoch: 01 | Time: 7m 2s
	Train Loss: 5.356 | Train PPL: 211.978
	 Val. Loss: 2.900 |  Val. PPL:  18.170
0 steps are done
1000 steps are done
2000 steps are done
Epoch: 02 | Time: 6m 50s
	Train Loss: 4.628 | Train PPL: 102.267
	 Val. Loss: 2.632 |  Val. PPL:  13.899
0 steps are done
1000 steps are done
2000 steps are done
Epoch: 03 | Time: 6m 53s
	Train Loss: 3.964 | Train PPL:  52.691
	 Val. Loss: 2.632 |  Val. PPL:  13.903
0 steps are done
1000 steps are done
2000 steps are done
Epoch: 04 | Time: 7m 5s
	Train Loss: 3.678 | Train PPL:  39.569
	 Val. Loss: 2.466 |  Val. PPL:  11.777
0 steps are done
1000 steps are done
2000 steps are done
Epoch: 05 | Time: 7m 15s
	Train Loss: 3.438 | Train PPL:  31.113
	 Val. Loss: 2.401 |  Val. PPL:  11.030
0 steps are done
1000 steps are done
2000 steps are done
Epoch: 06 | Time: 7m 6s
	Train Loss: 3.204 | Train PPL:  24.627
	 Val. Loss: 2.380 |  Val. PPL:  10.801
0 steps are done
1000 steps are

Finally, we test the model on the test set using these "best" parameters.

In [48]:
model.load_state_dict(torch.load('squad-attention.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 2.257 | Test PPL:   9.551 |
